In [1]:
import os, random, json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import SequentialSampler, SubsetRandomSampler
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader as GeoDataLoader

import optuna
from metrics import metrics_from_preds
from preprocess import *

# Carga de datos

In [2]:
"""
# Para secuenciales
train_loader_seq = DataLoader(VideoDataset(vids_training, labels_training), batch_size=32, shuffle=False)
val_loader_seq   = DataLoader(VideoDataset(vids_val, labels_val), batch_size=32, shuffle=False)
"""

'\n# Para secuenciales\ntrain_loader_seq = DataLoader(VideoDataset(vids_training, labels_training), batch_size=32, shuffle=False)\nval_loader_seq   = DataLoader(VideoDataset(vids_val, labels_val), batch_size=32, shuffle=False)\n'

In [3]:
def set_seed(seed=42):
    SEED = seed
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Controla el hash aleatorio de Python
    os.environ["PYTHONHASHSEED"] = str(seed)

In [4]:
set_seed(42)

In [5]:
device = torch.device("cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cpu


In [6]:
vids_training, labels_training, vids_val, labels_val, vids_test, labels_test = load_data()

In [7]:
def build_graph(frames, label):
    """
    Convierte un video (16 x 258) en un grafo:
    - 16 nodos (uno por frame).
    - Cada nodo con 258 features.
    - Aristas entre frames consecutivos.
    """
    x = torch.tensor(frames, dtype=torch.float32)  # (16, 258)

    # Conexiones secuenciales (cadena temporal)
    edge_index = []
    for i in range(len(frames) - 1):
        edge_index.append([i, i+1])
        edge_index.append([i+1, i])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()  # (2, E)

    y = torch.tensor([label], dtype=torch.long)  # etiqueta del grafo
    return Data(x=x, edge_index=edge_index, y=y)

In [8]:
class VideoGraphDataset(InMemoryDataset):
    def __init__(self, videos, labels, transform=None):
        self.videos = videos
        self.labels = labels
        super().__init__('.', transform)

        self.data_list = [build_graph(v, l) for v, l in zip(videos, labels)]

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        return self.data_list[idx]

In [9]:
set_seed()

train_graph_dataset = VideoGraphDataset(vids_training, labels_training)
val_graph_dataset   = VideoGraphDataset(vids_val, labels_val)


# Permutación fija para tener "shuffle" pero reproducible
g = torch.Generator().manual_seed(42)
perm = torch.randperm(len(train_graph_dataset), generator=g).tolist()
train_sampler = SubsetRandomSampler(perm)

train_loader_graph = GeoDataLoader(train_graph_dataset, batch_size=32, shuffle=False, num_workers=0, sampler=train_sampler)
val_loader_graph   = GeoDataLoader(val_graph_dataset, batch_size=32, shuffle=False, num_workers=0)

C:\Users\polar\AppData\Local\Temp\ipykernel_9940\1024000706.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  x = torch.tensor(frames, dtype=torch.float32)  # (16, 258)


# Modelo GCN (Baseline)

In [10]:
class GCNModel(nn.Module):
    """GCN simple para clasificación de 4 clases.
    Entrada: grafo de un frame/video -> salida: logits (B, 4)
    """
    def __init__(self, in_channels=258, hidden_channels=256, num_classes=4, layers=2, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        # Primer bloque
        self.layers = nn.ModuleList([GCNConv(in_channels, hidden_channels)])
        # Capas intermedias (layers - 1)
        for _ in range(layers - 1):
            self.layers.append(GCNConv(hidden_channels, hidden_channels))
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, batch):
        # x: nodos (N, in_channels), edge_index: aristas, batch: asignación de nodos a grafos
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        x = self.dropout(x)
        x = global_mean_pool(x, batch)  # pooling global
        return self.lin(x)

In [11]:
# ========= GCN =========
def optimize_gcn(train_loader_graph, val_loader_graph, n_trials=50, max_epochs=80, save_prefix="hpo_gcn"):
    set_seed()
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
    all_trials = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def objective(trial):
        hidden  = trial.suggest_categorical("hidden",[96,128,160,192])
        layers  = trial.suggest_int("layers", 2, 5)
        drop    = trial.suggest_float("dropout", 0.0, 0.5)
        lr      = trial.suggest_float("lr", 5e-5, 5e-3, log=True)
        wd      = trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)

        model = GCNModel(hidden_channels=hidden, layers=layers, dropout=drop).to(device)
        crit = nn.CrossEntropyLoss()
        opt  = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

        best_val = 0.0
        for epoch in range(max_epochs):
            model.train()
            for batch in train_loader_graph:
                batch = batch.to(device)
                opt.zero_grad()
                out = model(batch.x, batch.edge_index, batch.batch)
                loss = crit(out, batch.y)
                loss.backward(); opt.step()
            # val
            model.eval()
            y_true, y_pred = [], []
            with torch.no_grad():
                for batch in val_loader_graph:
                    batch = batch.to(device)
                    pred = model(batch.x, batch.edge_index, batch.batch).argmax(1)
                    y_true.extend(batch.y.cpu().numpy()); y_pred.extend(pred.cpu().numpy())
            val_acc = metrics_from_preds(y_true, y_pred)["accuracy"]
            if val_acc > best_val: best_val = val_acc
            trial.report(val_acc, epoch)
            if trial.should_prune(): raise optuna.TrialPruned()

        all_trials.append({"trial": trial.number, "params": trial.params, "best_val_acc": best_val})
        return best_val

    study.optimize(objective, n_trials=n_trials)
    with open(f"{save_prefix}_all.json","w") as f: json.dump(all_trials,f,indent=2)
    with open(f"{save_prefix}_best.json","w") as f: json.dump({"best_params":study.best_params,
                                                               "best_value":study.best_value},
                                                              f,
                                                              indent=2)
    return study

In [12]:
study_gcn = optimize_gcn(train_loader_graph, val_loader_graph, n_trials=40, max_epochs=100)

[I 2025-11-30 21:06:41,585] A new study created in memory with name: no-name-30066038-5cfb-43ff-a810-87007d568186
[I 2025-11-30 21:07:14,552] Trial 0 finished with value: 0.8277777777777777 and parameters: {'hidden': 160, 'layers': 4, 'dropout': 0.011069122058494585, 'lr': 5.782093145482609e-05, 'weight_decay': 1.22648131364617e-08}. Best is trial 0 with value: 0.8277777777777777.
[I 2025-11-30 21:07:39,192] Trial 1 finished with value: 0.8444444444444444 and parameters: {'hidden': 96, 'layers': 5, 'dropout': 0.2522414346538968, 'lr': 0.0019201533554539072, 'weight_decay': 2.3791026092081835e-09}. Best is trial 1 with value: 0.8444444444444444.
[I 2025-11-30 21:08:07,311] Trial 2 finished with value: 0.8444444444444444 and parameters: {'hidden': 128, 'layers': 3, 'dropout': 0.4988993420902897, 'lr': 0.0023990659904421147, 'weight_decay': 7.798400040517562e-10}. Best is trial 1 with value: 0.8444444444444444.
[I 2025-11-30 21:08:40,271] Trial 3 finished with value: 0.8666666666666667 an

In [14]:
print("Mejor resultado:", study_gcn.best_value)
print("Mejores hiperparámetros:", study_gcn.best_params)

Mejor resultado: 0.8833333333333333
Mejores hiperparámetros: {'hidden': 160, 'layers': 4, 'dropout': 0.19636922365934475, 'lr': 0.001313660897462963, 'weight_decay': 1.0182638526745577e-05}


In [15]:
import json
from metrics import metrics_from_preds
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import global_mean_pool

# --- Cargar los mejores hiperparámetros ---
print("Cargando los mejores hiperparámetros desde hpo_gcn_best.json...")
try:
    with open("hpo_gcn_best.json", "r") as f:
        best_hyperparams = json.load(f)["best_params"]
    print("Hiperparámetros cargados:")
    print(best_hyperparams)
except FileNotFoundError:
    print("Error: El archivo 'hpo_gcn_best.json' no fue encontrado.")
    print("Por favor, ejecuta primero la celda de optimización para generar este archivo.")
    raise

# --- Configurar y entrenar el mejor modelo ---
set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instanciar el modelo con los mejores hiperparámetros
# Nota: El modelo GCN usa 'hidden_channels', que corresponde a 'hidden' en el JSON.
best_model = GCNModel(
    hidden_channels=best_hyperparams["hidden"],
    layers=best_hyperparams["layers"],
    dropout=best_hyperparams["dropout"]
).to(device)

# Definir criterio de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(
    best_model.parameters(),
    lr=best_hyperparams["lr"],
    weight_decay=best_hyperparams["weight_decay"]
)

# Entrenamiento del modelo
epochs = 100  # Usamos las mismas épocas que en la búsqueda de hiperparámetros
print(f"\nEntrenando el mejor modelo GCN durante {epochs} épocas...")

for epoch in range(epochs):
    best_model.train()
    total_loss = 0
    for batch in train_loader_graph:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = best_model(batch.x, batch.edge_index, batch.batch)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader_graph):.4f}")

print("Entrenamiento completado.")

# --- Evaluación del modelo y cálculo de métricas ---
best_model.eval()
all_metrics = {}

def evaluate_graph_model(loader, model_name):
    y_true, y_pred = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            predictions = best_model(batch.x, batch.edge_index, batch.batch).argmax(1)
            y_true.extend(batch.y.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())
    
    metrics = metrics_from_preds(y_true, y_pred)
    all_metrics[model_name] = metrics

print("\n--- Métricas del Modelo Final (GCN) ---")
evaluate_graph_model(train_loader_graph, "Train")
evaluate_graph_model(val_loader_graph, "Validation")

for split_name, metrics in all_metrics.items():
    print(f"\nResultados en el conjunto de {split_name}:")
    for metric_name, value in metrics.items():
        if isinstance(value, float):
            print(f"  {metric_name}: {value:.4f}")
        else:
            print(f"  {metric_name}: {value}")

# --- Guardar el modelo entrenado ---
model_save_path = "gcn_best_model.pth"
torch.save(best_model.state_dict(), model_save_path)
print(f"\nModelo guardado exitosamente en: {model_save_path}")

Cargando los mejores hiperparámetros desde hpo_gcn_best.json...
Hiperparámetros cargados:
{'hidden': 160, 'layers': 4, 'dropout': 0.19636922365934475, 'lr': 0.001313660897462963, 'weight_decay': 1.0182638526745577e-05}

Entrenando el mejor modelo GCN durante 100 épocas...
Epoch [10/100], Loss: 0.6362
Epoch [20/100], Loss: 0.3947
Epoch [30/100], Loss: 0.3190
Epoch [40/100], Loss: 0.2909
Epoch [50/100], Loss: 0.2539
Epoch [60/100], Loss: 0.1835
Epoch [70/100], Loss: 0.1957
Epoch [80/100], Loss: 0.1660
Epoch [90/100], Loss: 0.1712
Epoch [100/100], Loss: 0.1188
Entrenamiento completado.

--- Métricas del Modelo Final (GCN) ---

Resultados en el conjunto de Train:
  accuracy: 0.9804
  precision_macro: 0.9805
  recall_macro: 0.9804
  specificity_macro: 0.9935
  f1_macro: 0.9804
  balanced_accuracy: 0.9804
  confusion_matrix: [[253, 0, 1, 1], [0, 246, 7, 2], [0, 3, 252, 0], [2, 3, 1, 249]]

Resultados en el conjunto de Validation:
  accuracy: 0.8389
  precision_macro: 0.8688
  recall_macro: 0